# Six-bar mechanism balancing

###### Luis Eduardo Robles Jiménez

With this work, I pretend to find out the behavior of the function with different tools, in order to understand what the most important variables are, find patterns, identify overlapping zones of minima between the functions we're optimizing, graphing data, etc.

In [ ]:
from BetaShF import *
from BetaShM import *
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import plotly.express as px
from sympy import *
import numpy as np
import scipy as sp
import time


assert ShF([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0
assert ShM([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) == 1.0

#### Constraints

$$ -0.16m \leq x_{cn}, y_{cn} \leq 0.16m $$

$$ 0.005m \leq t_{cn} \leq 0.04m $$

#### Objective function

In [ ]:
def objective(s, ShF, ShM):
    alpha = np.random.uniform()
    return (alpha)*ShF(s) + (1 - alpha)*ShM(s)

## Mathematical analysis 

### Sympy

In [ ]:
from momentFormula import *
from forceFormula import *

In [ ]:
a = srepr(forceF)
b = srepr(momentF)

## Sample generation

In [ ]:
nSamples, nVariables = 300, 15
nWeights = nVariables // 3
samples = np.zeros((nSamples, nVariables))
bounds = np.zeros((nVariables, 2))
fitness = np.zeros((nSamples))
shForces = np.zeros((nSamples))
shMoments = np.zeros((nSamples))

### Differential Evolution

In [ ]:
"""
for i in range(nVariables):
    if (i + 1) % 3 == 0: samples[:, i] = np.random.uniform(0.005, 0.04, (nSamples))
    else: samples[:, i] = np.random.uniform(-0.16, 0.16, (nSamples))
for i in range(nSamples):
    fitness[i] = objective(samples[i], ShF, ShM)
"""
for v in range(nVariables):
    bounds[v] = [0.005, 0.04] if (v + 1) % 3 == 0 else [-0.16, 0.16]
worst = 0.0
for s in range(nSamples):
    print("\rIteration:", s, "\tWorst fitness:", worst, end = "")
    p = False
    #if s < nSamples // 10: p = True
    r = sp.optimize.differential_evolution(objective, bounds, args = (ShF, ShM), maxiter = 100, popsize = 50, polish = p)
    samples[s] = r.x
    fitness[s] = r.fun
    worst = np.max([worst, fitness[s]])

### Pymoo

In [ ]:
from pymoo.core.problem import ElementwiseProblem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

In [ ]:
class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=nVariables,
                         n_obj=2,
                         n_ieq_constr=0,
                         xl=np.array([-0.16, -0.16, 0.005, -0.16, -0.16, 0.005, -0.16, -0.16, 0.005, -0.16, -0.16, 0.005, -0.16, -0.16, 0.005]),
                         xu=np.array([0.16, 0.16, 0.04, 0.16, 0.16, 0.04, 0.16, 0.16, 0.04, 0.16, 0.16, 0.04, 0.16, 0.16, 0.04]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = ShF(x)
        f2 = ShM(x)

        out["F"] = [f1, f2]
        out["G"] = []


problem = MyProblem()

In [ ]:
algorithm = NSGA2(
    pop_size=500,
    n_offsprings=500,
    sampling=FloatRandomSampling(),
    crossover=UniformCrossover(prob=0.75),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [ ]:
termination = get_termination("n_gen", 1000)

In [ ]:
res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

X = res.X
F = res.F

In [ ]:
print(X.shape)
print(F.shape)
samples = np.copy(X)
fitness = np.average(F, axis = 1)
shForces = F[:, 0]
shMoments = F[:, 1]

In [ ]:
plt.figure(figsize=(7, 5))
plt.scatter(F[:, 0], F[:, 1], s=30, facecolors='none', edgecolors='blue')
plt.title("Objective Space")
plt.show()

### Clean data

In [ ]:
filterF = np.where(shForces < 1)
filterM = np.where(shMoments < 1)
f = np.logical_and(filterF, filterM)
print(f.shape)
samples = samples[f]
fitness = fitness[f]
shForces = shForces[f]
shMoments = shMoments[f]

### Write data to a file

In [ ]:
now = str(int(time.time()))
np.savetxt(now + "Population.txt", samples)
np.savetxt(now + "Fitness.txt", fitness)
np.savetxt(now + "ShForces.txt", shForces)
np.savetxt(now + "ShMoments.txt", shMoments)